In [9]:
#| default_exp routes.storage

# Storage Routes
> Default description (change me)

In [1]:
#| export
from supabase import create_client
from supabase import Client as SupabaseClient


import agent_mafia.utils.files as amfi
import json
import datetime as dt

from agent_mafia.client.ResponseGetData import ResponseGetDataSupabase

In [2]:
#| hide
import os
import nbdev

In [3]:

supabase_client : SupabaseClient = create_client(
    os.environ["SUPABASE_URL"], 
    os.environ["SUPABASE_SERVICE_KEY"]
)

In [4]:
#| export
def store_data_in_supabase_table(
    supabase_client: SupabaseClient,
    table_name : str,
    data: dict,
):
    """stores data into a supabase table"""
    res = (
        supabase_client.table(table_name)
        .upsert(data, on_conflict="url, chunk_number")
        .execute()
    )

    rgd = ResponseGetDataSupabase.from_res(res = res)

    return rgd

In [5]:
with open('../../TEST/storage_routes/test_upload.md' , 'r', encoding = 'utf-8') as f:
    data = f.read()
        
res = store_data_in_supabase_table(
    table_name = "site_pages",
    data = {
        "url" : "test.md",
        "chunk_number" : 0,
        "title": "Domo IT Whitepaper",
        "summary" : "a whitepaper a about IT",
        "content": data},
    supabase_client= supabase_client
)

res.response


[{'id': 2506,
  'url': 'test.md',
  'chunk_number': 0,
  'title': 'Domo IT Whitepaper',
  'summary': 'a whitepaper a about IT',
  'content': '# Executive Summary\n\nIT departments have invested heavily in people and systems to \xa0get closer to the fundamental goal of unlocking the right data, \xa0at the right time, for the right people. But to date, businesses \xa0have been forced to cobble together disparate efforts, with no \xa0single solution able to deliver the combination of functionality, \xa0security, and ease of use required to bring data efficiently to \xa0everyone within an organization. This paper explores what \xa0makes Domo’s solution different, how it solves common \xa0challenges faced by IT departments, and how Domo offers a \xa0data strategy built for enterprise but designed for people.\n\nLearn how Domo:\n\n- Functions as much more than just another BI or visualization tool. + Complements, rather than threatens, your existing infrastructure and systems. + Brings trust

In [6]:
# | export


def save_chunk_to_disk(
    output_path,
    data: dict,
    **kwargs,
):

    amfi.upsert_folder(output_path)

    url = data["url"]
    source = data["source"]
    content = data["content"]
    title = data.get("title")
    summary = data.get("summary")
    embedding = data.get("embedding")
    metadata = data.get("metadata")
    chunk_number = data.get("chunk_number")

    output_ls = [
        "---",
        f"url: {url}",
        f"session_id: {source}",
        f"chunk_number: {chunk_number}" if chunk_number is not None else None,
        f"title: {title}" if title is not None else None,
        f"summary: {summary}" if summary is not None else None,
        f"embedding: {embedding}" if embedding is not None else None,
        f"metadata : {json.dumps(metadata)}" if metadata is not None else None,
        f"updated_dt: {dt.datetime.now().isoformat()}",
        "---",
        content,
    ]

    with open(output_path, "w+", encoding="utf-8") as f:
        f.write("\n".join([row for row in output_ls if row is not None]))

        return True

In [7]:
save_chunk_to_disk(output_path = "../../TEST/storage_routes/save_to_disk.md",data = {"source" : 'test',**res.response[0], 
                                                                                     })

True

In [11]:
#| hide
nbdev.nbdev_export('/.storage.ipynb')